In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt

sys.path.insert(0, './TetrisEnv')
from TetrisBattle.envs.tetris_env import TetrisSingleEnv, TetrisEnv
from TetrisBattle import *
# %pip install -e ./TetrisEnv
# import TetrisBattle
%load_ext autoreload
%autoreload 2
from gym.wrappers import GrayScaleObservation
from preprocessing.EensyWeensy import MakeEensyWeensy
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# from gym import make
# env = make('SinglePTetris-v0')

pygame 2.5.2 (SDL 2.28.3, Python 3.10.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from gym import ActionWrapper
from gym.spaces import Discrete
class ReduceActions(ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        ### Action Space Mappings: ###
        # For shifting left and right #
        # using clockwise rotation (3) #
        # 6 left shifts at most #
        # 4 right shifts at most #
        self.ret_string = '0'
        self.action_space = Discrete(44)

    def shift(self, code):
        # recenter the code to the center of the piece
        code -= 6

        # append to the action sequence
        for i in range(abs(code)):
            if code > 0:
                self.ret_string += '5'
            elif code < 0:
                self.ret_string += '6'
            else:
                pass
        return
    
    def rotate(self, code):
        match code:
            case 0: # keep current rotation
                pass
            case 1: # 1 cw
                self.ret_string += '03'
            case 2: # 2 cw
                self.ret_string += '0303'
            case 3: # 1 ccw
                self.ret_string += '04'
        return
    
    def action(self, act):
        # reset the return string
        self.ret_string = '0'

        # rotation code is the 11s place
        rotate_code = act // 11
        # shift code is the 1s place
        shift_code = act % 11

        self.rotate(rotate_code)
        self.shift(shift_code)

        # append the force drop to the sequence
        self.ret_string += '02'

        # print(self.ret_string)
        return self.ret_string


In [3]:
# env = TetrisSingleEnv()
# env = ReduceActions(env)
# env.reset()
# for i in range(1000):
#     action = int(input('>'))
#     # action = env.action_space.sample()
#     # print(action)
#     state, reward, done, info = env.step(action)
#     if done:
#         env.reset()
#     env.render()

# env.close()

In [4]:
env = TetrisSingleEnv()
env = GrayScaleObservation(env, keep_dim=True)
env = MakeEensyWeensy(env, cut_in_half=True, scale=.25)
env = ReduceActions(env)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order="last")
# ob = env.reset()
# plt.imshow(ob, cmap="plasma")

In [5]:
import os 
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.env_checker import check_env
import torch
print(torch.cuda.is_available())
# check_env(env)

True


In [6]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [7]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [8]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [9]:
try:
    del model
except NameError:
    pass
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=1e-4, n_steps=64, device="cuda") 
# model = PPO.load('./train/cams100000')
# model.set_env(env)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [10]:
# model.learn(total_timesteps=1000000, callback=callback)

In [11]:
model = PPO.load('./train/best_model_90000')
env = TetrisSingleEnv()
env = GrayScaleObservation(env, keep_dim=True)
env = MakeEensyWeensy(env, cut_in_half=True, scale=.25)
env = ReduceActions(env)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order="last")
state = env.reset()

for i in range(0, 10000):
    action, _ = model.predict(state)
    # action = env.action_space.sample()
    # print(action)
    state, reward, done, info = env.step(action)
    env.render()

env.close()

In [ ]:
# class InputGetter():
#     def __init__(self):
#         self.input_log = []
        
#     def get_input(self):
#         while True:
#             try:
#                 if len(self.input_log) == 0:
#                     self.input_log = [*input(">")]
                
#                 inp = self.input_log.pop(0)
#                 inp = int(inp)
#                 assert(inp in [0, 1, 2, 3, 4, 5, 6, 7, 9])
#                 return inp
#             except Exception as e:
#                 print(e)
# env = TetrisSingleEnv(obs_type="grid")
# i = 0
# inputter = InputGetter()
# while True:
#     action = inputter.get_input()
#     if action == 9:
#         env.close()
#         break
#     state, reward, done, info = env.step(action)
#     # print("infow", info)
#     # print("rewrd",reward)
#     env.render()
#     # print(env.game_interface.get_obs().shape)
#     grid = env.game_interface.get_obs()